# PyTorch Deconvolutional Autoencoder Lab

### Why this notebook
- Demonstrate the convolutional (a.k.a. deconvolutional) autoencoder built in `../src`.
- Provide a recipe for training, evaluating, and visualising convolutional reconstructions.
- Highlight how this architecture differs from the dense vanilla baseline.

### Learning objectives
- Train the convolutional autoencoder on Fashion-MNIST with the modular PyTorch package.
- Reconstruct samples and compare spatial detail against the MLP variant.
- Experiment with stride, kernel size, and channel depth modifications.

### Prerequisites
- PyTorch 2.x, torchvision, and Matplotlib installed.
- Familiarity with the vanilla autoencoder workflow.
- Optional: GPU/MPS for efficient training.

### Notebook workflow
1. Import config, training, and inference helpers from `../src`.
2. Execute `train(CONFIG)` and inspect reconstruction metrics.
3. Load checkpoints and visualise original vs reconstructed images.
4. Extend with convolutional architecture tweaks, feature map inspections, or denoising experiments.


**Workflow**

1. Import the package and view the configuration.
2. Train the autoencoder (automatic `mps`/`cuda`/`cpu` selection).
3. Reconstruct a sample image to verify the decoder.

In [ ]:
from pathlib import Path
import sys

NOTEBOOK_DIR = Path().resolve()
SRC_DIR = NOTEBOOK_DIR.parent / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from config import CONFIG  # noqa: E402
from inference import load_model, reconstruct  # noqa: E402
from train import train  # noqa: E402

CONFIG

In [ ]:
metrics = train(CONFIG)
metrics

### Interpret the metrics
- Reconstruction loss and PSNR track how well the convolutional decoder restores details.
- Compare these curves with the vanilla autoencoder to quantify benefits of conv layers.
- Track validation metrics separately to catch overfitting due to high capacity.
- Use TensorBoard integration for longer training runs.

In [ ]:
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
test_ds = datasets.FashionMNIST(root=str(CONFIG.data_dir), train=False, download=True, transform=transform)
image, _ = test_ds[0]
model = load_model(config=CONFIG)
reconstruction = reconstruct([image], model=model, config=CONFIG)[0]

def to_numpy(tensor):
    return tensor.squeeze().cpu().numpy() * 0.5 + 0.5

fig, axes = plt.subplots(1, 2, figsize=(6, 3))
axes[0].imshow(to_numpy(image), cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')
axes[1].imshow(to_numpy(reconstruction), cmap='gray')
axes[1].set_title('Reconstruction')
axes[1].axis('off')
plt.tight_layout()

### Next experiments
- Visualise intermediate feature maps to understand what the encoder captures.
- Add skip connections (U-Net style) and compare reconstruction sharpness.
- Introduce noise to inputs and test denoising performance without re-training.
- Benchmark against the TensorFlow implementation to confirm architectural parity.